Instalação e importação de módulos

In [1]:
!pip install -q transformers torch accelerate

In [2]:
import os
import torch
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import clear_output
from google.colab import drive

Antes de prosseguir, verificar se está com a GPU habilitada. A inferência demora demais se estiver usando apenas CPU.

In [19]:
print("GPU disponível?", torch.cuda.is_available())

if not torch.cuda.is_available():
    print("Ative a GPU em Runtime -> Change runtime type -> T4 GPU.")

GPU disponível? True


Carregando o modelo Qwen2.5-Coder-1.5B-Instruct do HuggingFace

In [4]:
model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

print(f"Carregando {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float16,
    device_map="cuda"
)

Carregando Qwen/Qwen2.5-Coder-1.5B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Definição de função de inferência do modelo a partir do prompt.

In [20]:
def inferir_arquitetura_pela_tree(prompt):
    messages = [
        {"role": "system", "content": "Você é um especialista em análise de código e padrões arquiteturais."},
        {"role": "user", "content": prompt}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=True, return_dict=True, return_tensors="pt", add_generation_prompt=True).to(model.device)

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.2
    )

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].split("assistant")[-1].strip()

Definição das entradas


In [21]:
input = []

for idx in range(1,5):
  with open(f"./inputs/entrada{idx}.txt", 'r', encoding="utf-8") as f:
    input.append(f.read())

Montagem do prompt

In [22]:
prompt = """
Act as a Senior Software Architect.

Analyze the following software repository artifacts:
1. The folder structure (File Tree);
2. The package.json file;
3. The commit history (last 100 commits).

TASK:
Identify which architectural patterns are present in the analyzed repository.
(Examples of architectural patterns: Layered Architecture, Client-Server, Microservices, Monolithic, Pipe-and-Filters, Event-Driven Architecture, Service-Oriented Architecture (SOA), Peer-to-Peer (P2P), Blackboard, Hexagonal Architecture (Ports and Adapters), Clean Architecture, Serverless, MVC (Model-View-Controller), CQRS, Microkernel (Plug-ins), Broker Pattern, Master-Slave).

Provide 5 predictions with associated probability percentages.

STRICTLY follow this format:
1. [Pattern Name] - [XX]%
   Justification: [Explanation for the prediction based on the artifacts]

--- START OF TREE ---
{}
--- END OF TREE ---

--- START OF PACKAGE.JSON ---
{}
--- END OF PACKAGE.JSON ---

--- START OF COMMIT HISTORY ---
{}
--- END OF COMMIT HISTORY ---
"""

Por fim, executamos o modelo e esperamos pelo resultado da análise (aprox. 40s)

In [23]:
repo_path = "./anything-llm"

print("Aguardando análise do modelo...")

start = time.perf_counter()
resultado = inferir_arquitetura_pela_tree(prompt.format(input[3], input[1], input[2]))
end = time.perf_counter()

tempo_execucao = end-start

clear_output(wait=True)

print("Análise gerada:\n")
print(f"Tempo de execução = {tempo_execucao}s\n")

with open(f"output.txt", 'w') as f:
  f.write(resultado)


Análise gerada:

Tempo de execução = 22.217738627000017s



Fazer download das entradas e saídas

In [24]:
from google.colab import files

files.download("output.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>